# Realizando ingeniería de datos para realizar transformación y generación de nuevas variables

## 1.- Importamos las librerías necesarias

In [3]:
import pandas as pd
import numpy as np
import os

### Obtenemos los datos guardados.

In [6]:
path='Data\scrapped_data.csv'

In [8]:
os.chdir(os.path.split(os.getcwd())[0])
folder=os.getcwd()
folder

'd:\\Data_science\\Javier\\Repositorios\\Proyecto_tienda_online'

In [11]:
new_path=folder+'\\'+path
new_path

'd:\\Data_science\\Javier\\Repositorios\\Proyecto_tienda_online\\Data\\scrapped_data.csv'

In [12]:
dataframe=pd.read_csv(new_path)
dataframe

,Name,Subname,Description,Characteristics,Price,Reduced Price,date,User,Ratings,Comment
0,FOXTAIL,['plug anal cola de zorro de 35cm'],['Deja volar la imaginación y saca tu lado más...,"['', '', 'Tamaño plug S: 6.6cm Max Dia.: 2.8cm...",21.99,9.99,"['jueves 26 enero, 2023', 'jueves 05 enero, 20...","['Jonatan', 'Irene', 'Ainara', 'Ignacio', 'and...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 4]","['Un buen plug, muy suave, aunque con el lubri..."
1,LIZO 2,['Dildo de suave silicona en 3 tamaños'],"['En un azulejo de la cocina, en una puerta, e...",['Tanto para tus momentos de onanismo como par...,21.99,17.99,"['domingo 12 marzo, 2023', 'martes 10 enero, 2...","['Iria', 'Barney', 'Sara', 'Aida', 'Lucas', 'a...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...",['Al no tener tope viene perfecto para usar co...
2,TOBOGANE HOT RABBIT,['el superventas de amantis ¡mejorado!'],['Vuelve nuestro vibrador de doble estimulació...,"['', '', 'Medidas: 19cm (11cm insertables) y 3...",21.99,39.99,"['martes 22 noviembre, 2022', 'jueves 07 julio...","['Rossi', 'Marina', 'Jennifer', 'Noa', 'Karen'...","[5, 5, 5, 4, 3, 5, 5, 5, 5, 5, 4, 5, 5, 3, 5, ...","['Mi primera compra. Me encantó la textura, lo..."
3,TOBOGANE,['el vibrador doble más vendido'],"['Por favor, desabróchense los cinturones de s...","['', 'La última atracción exclusiva de amantis...",21.99,36.99,"['lunes 06 marzo, 2023', 'sábado 04 marzo, 202...","['Teresa', 'Alicia', 'María', 'Sara', 'Raquel'...","[5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 5, ...",['Fue el primer vibrador que compré. Me lo rec...
4,MENEO sube y baja,['placer realista con control remoto'],"['Si te gusta que te metan un buen meneo, hazt...","['Ya lo ves, Meneo’s Cock puede ir contigo a d...",21.99,44.99,"['miércoles 14 diciembre, 2022', 'miércoles 09...","['Francisco', 'Maria', 'Jose Javier', 'Carlos'...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4]",['Quería saber cuantos cm tiene la longitud qu...
...,...,...,...,...,...,...,...,...,...,...
571,JUSTISSE NECK - collar + bloqueadores de muñecas,['No hay datos'],"['Muy a menudo, las parejas se quejan de que l...","['\r', 'Con este kit JUSTISSE NECK solo tienes...",21.99,23.99,"['domingo 08 noviembre, 2020']",['Alejandro'],[5],['Es un poco engorroso a la hora de ponerlo so...
572,MS-VULVAN. Estimulador Total de Vulva (Cabezal...,['No hay datos'],['La orgásmica colección de accesorios para ac...,"['\r', 'Realizados en un agradable TPR liso y ...",21.99,8.99,"['jueves 02 agosto, 2018', 'martes 17 julio, 2...","['Inma', 'yulema', 'Cristina', 'Veturian', 'Pi...","[5, 5, 3, 5, 5, 4, 3]","['Es uno de mis cabezales estrella, es muy com..."
573,AVATAR SIMON PRIDE,['Orgulloso dildo de silicona'],['¿Quién dijo que en el mundo de los dildos no...,"['', 'Juguete de silicona 100%', 'Dimensiones:...",21.99,34.99,"['martes 06 agosto, 2019']",['Javier'],[4],['A mi chica no le gustaba la típica aparienci...
574,TSUNAMI FORMAS,['Fundas de silicona para TSUNAMI POWER'],['¿Has echado un vistazo a nuestro productazo ...,"['', '', 'Un dildo con anilla, porque hay vece...",21.99,14.99,"['miércoles 11 noviembre, 2020', 'miércoles 01...","['Christian', 'Carmen', 'Meritxell', 'Esther',...","[5, 5, 4, 5, 5]",['Imprescindibles para disfrutar del Tsunami c...
